In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'diabetes'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.058824,0.232394,0.488372,0.285714,0.096154,0.202454,0.035118,0.000000,0
1,0.000000,0.570423,0.186047,0.500000,0.185096,0.509202,0.943469,0.200000,1
2,0.176471,0.154930,0.302326,0.446429,0.088942,0.261759,0.069807,0.083333,1
3,0.117647,0.992958,0.534884,0.678571,0.635817,0.251534,0.031263,0.533333,1
4,0.058824,0.936620,0.418605,0.285714,1.000000,0.243354,0.134047,0.633333,1


In [5]:
target = 'Outcome'
y = df[target].values
del df[target]
x = df.values

In [6]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [7]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [8]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [9]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [10]:
num_epochs = 500

In [11]:
x.shape[1]

8

In [12]:
latent_dims = 4
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [13]:
latent_dims

4

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [15]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 94.087765
Epoch [2 / 500] average loss: 76.891514
Epoch [3 / 500] average loss: 62.607225
Epoch [4 / 500] average loss: 53.651886
Epoch [5 / 500] average loss: 45.297170
Epoch [6 / 500] average loss: 39.670828
Epoch [7 / 500] average loss: 34.067425
Epoch [8 / 500] average loss: 29.409100
Epoch [9 / 500] average loss: 25.630804
Epoch [10 / 500] average loss: 22.231403
Epoch [11 / 500] average loss: 19.233039
Epoch [12 / 500] average loss: 17.300936
Epoch [13 / 500] average loss: 14.956759
Epoch [14 / 500] average loss: 13.981554
Epoch [15 / 500] average loss: 13.083639
Epoch [16 / 500] average loss: 11.956169
Epoch [17 / 500] average loss: 11.309686
Epoch [18 / 500] average loss: 10.159001
Epoch [19 / 500] average loss: 9.823418
Epoch [20 / 500] average loss: 9.621244
Epoch [21 / 500] average loss: 9.283013
Epoch [22 / 500] average loss: 8.886392
Epoch [23 / 500] average loss: 8.918139
Epoch [24 / 500] average loss: 8.603756
Epoch [25 / 500] average lo

Epoch [205 / 500] average loss: 6.994875
Epoch [206 / 500] average loss: 6.991092
Epoch [207 / 500] average loss: 7.006383
Epoch [208 / 500] average loss: 6.989021
Epoch [209 / 500] average loss: 6.939156
Epoch [210 / 500] average loss: 6.976194
Epoch [211 / 500] average loss: 6.999827
Epoch [212 / 500] average loss: 7.025894
Epoch [213 / 500] average loss: 7.004541
Epoch [214 / 500] average loss: 7.027326
Epoch [215 / 500] average loss: 6.991497
Epoch [216 / 500] average loss: 7.007574
Epoch [217 / 500] average loss: 7.005449
Epoch [218 / 500] average loss: 6.967395
Epoch [219 / 500] average loss: 6.974647
Epoch [220 / 500] average loss: 6.981729
Epoch [221 / 500] average loss: 6.972213
Epoch [222 / 500] average loss: 7.003578
Epoch [223 / 500] average loss: 6.997612
Epoch [224 / 500] average loss: 7.013003
Epoch [225 / 500] average loss: 6.967069
Epoch [226 / 500] average loss: 7.008553
Epoch [227 / 500] average loss: 6.985002
Epoch [228 / 500] average loss: 7.032698
Epoch [229 / 500

Epoch [406 / 500] average loss: 6.964736
Epoch [407 / 500] average loss: 6.987757
Epoch [408 / 500] average loss: 7.001782
Epoch [409 / 500] average loss: 6.992773
Epoch [410 / 500] average loss: 6.977242
Epoch [411 / 500] average loss: 6.974951
Epoch [412 / 500] average loss: 7.001963
Epoch [413 / 500] average loss: 6.989541
Epoch [414 / 500] average loss: 6.981914
Epoch [415 / 500] average loss: 6.974414
Epoch [416 / 500] average loss: 6.982668
Epoch [417 / 500] average loss: 6.998791
Epoch [418 / 500] average loss: 6.979664
Epoch [419 / 500] average loss: 6.973581
Epoch [420 / 500] average loss: 6.992556
Epoch [421 / 500] average loss: 6.987624
Epoch [422 / 500] average loss: 6.966937
Epoch [423 / 500] average loss: 6.993629
Epoch [424 / 500] average loss: 6.988814
Epoch [425 / 500] average loss: 6.988781
Epoch [426 / 500] average loss: 6.970949
Epoch [427 / 500] average loss: 7.039017
Epoch [428 / 500] average loss: 7.005537
Epoch [429 / 500] average loss: 6.995292
Epoch [430 / 500

In [16]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

3
4


In [17]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [18]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 18.947808
Epoch [2 / 500] average reconstruction error: 12.177536
Epoch [3 / 500] average reconstruction error: 8.377435
Epoch [4 / 500] average reconstruction error: 6.023315
Epoch [5 / 500] average reconstruction error: 4.291304
Epoch [6 / 500] average reconstruction error: 3.122437
Epoch [7 / 500] average reconstruction error: 2.174729
Epoch [8 / 500] average reconstruction error: 1.593444
Epoch [9 / 500] average reconstruction error: 1.089333
Epoch [10 / 500] average reconstruction error: 0.765449
Epoch [11 / 500] average reconstruction error: 0.551568
Epoch [12 / 500] average reconstruction error: 0.454311
Epoch [13 / 500] average reconstruction error: 0.358635
Epoch [14 / 500] average reconstruction error: 0.304506
Epoch [15 / 500] average reconstruction error: 0.243259
Epoch [16 / 500] average reconstruction error: 0.198054
Epoch [17 / 500] average reconstruction error: 0.159432
Epoch [18 / 500] average reconstruction error: 0.13

Epoch [147 / 500] average reconstruction error: 0.003741
Epoch [148 / 500] average reconstruction error: 0.003671
Epoch [149 / 500] average reconstruction error: 0.003674
Epoch [150 / 500] average reconstruction error: 0.003525
Epoch [151 / 500] average reconstruction error: 0.003664
Epoch [152 / 500] average reconstruction error: 0.003639
Epoch [153 / 500] average reconstruction error: 0.003725
Epoch [154 / 500] average reconstruction error: 0.003800
Epoch [155 / 500] average reconstruction error: 0.003753
Epoch [156 / 500] average reconstruction error: 0.004011
Epoch [157 / 500] average reconstruction error: 0.003484
Epoch [158 / 500] average reconstruction error: 0.003742
Epoch [159 / 500] average reconstruction error: 0.003693
Epoch [160 / 500] average reconstruction error: 0.003813
Epoch [161 / 500] average reconstruction error: 0.003692
Epoch [162 / 500] average reconstruction error: 0.003628
Epoch [163 / 500] average reconstruction error: 0.003777
Epoch [164 / 500] average recon

Epoch [291 / 500] average reconstruction error: 0.003471
Epoch [292 / 500] average reconstruction error: 0.003410
Epoch [293 / 500] average reconstruction error: 0.003466
Epoch [294 / 500] average reconstruction error: 0.003317
Epoch [295 / 500] average reconstruction error: 0.003102
Epoch [296 / 500] average reconstruction error: 0.003106
Epoch [297 / 500] average reconstruction error: 0.003158
Epoch [298 / 500] average reconstruction error: 0.003185
Epoch [299 / 500] average reconstruction error: 0.003351
Epoch [300 / 500] average reconstruction error: 0.003133
Epoch [301 / 500] average reconstruction error: 0.003427
Epoch [302 / 500] average reconstruction error: 0.003538
Epoch [303 / 500] average reconstruction error: 0.003276
Epoch [304 / 500] average reconstruction error: 0.003366
Epoch [305 / 500] average reconstruction error: 0.003428
Epoch [306 / 500] average reconstruction error: 0.003311
Epoch [307 / 500] average reconstruction error: 0.003249
Epoch [308 / 500] average recon

Epoch [435 / 500] average reconstruction error: 0.003380
Epoch [436 / 500] average reconstruction error: 0.003136
Epoch [437 / 500] average reconstruction error: 0.003350
Epoch [438 / 500] average reconstruction error: 0.003154
Epoch [439 / 500] average reconstruction error: 0.003115
Epoch [440 / 500] average reconstruction error: 0.003309
Epoch [441 / 500] average reconstruction error: 0.003128
Epoch [442 / 500] average reconstruction error: 0.003215
Epoch [443 / 500] average reconstruction error: 0.003283
Epoch [444 / 500] average reconstruction error: 0.003174
Epoch [445 / 500] average reconstruction error: 0.003553
Epoch [446 / 500] average reconstruction error: 0.003692
Epoch [447 / 500] average reconstruction error: 0.003263
Epoch [448 / 500] average reconstruction error: 0.003397
Epoch [449 / 500] average reconstruction error: 0.003127
Epoch [450 / 500] average reconstruction error: 0.003371
Epoch [451 / 500] average reconstruction error: 0.003276
Epoch [452 / 500] average recon